# IDC ETL, v10
This notebook implements the IDC ETL process. It closely follows the steps in section 6 of [ETL Workflow, v10](https://docs.google.com/document/d/1luEnT0Vr5_VZwOYl2WaIwwHDvVfm0IBeXVzxzRGfzMs/edit#heading=h.2r0uhxc). Refer to that document for a description of the process.

## Preliminary

We do ETL development in Pycharm, wherein Pycharm is configured to execute ETL scripts on a remote VM. For purposes of remote execution, Pycharm maintains a copy of most of the files in a Pycharm project on the remote VM.
Presumably, one could also pull/clone the project data from the [etl_flow](https://github.com/ImagingDataCommons/etl_flow) repo. We have not yet tried that.

When performing the ingestion step, an 8-core VM is recommended to support multi-process downloading of the data. For other tasks, a 2-core VM is usually sufficient.

Regardless, the following constants should be change to define the location of the etl_flow project.

VM is an alias of the VM on which ETL scripts are executed. Such an alias can be generated by the [gcloud compute config-ssh](https://cloud.google.com/sdk/gcloud/reference/compute/config-ssh) CLI.

EF is the top directory of the Pycharm etl_flow project

In [182]:
VM = 'etl-dev-whc.us-central1-a.idc-etl-processing'
EF = '/pycharm/etl_flow/tmp/pycharm_project_936'

Define an alias for initiating remote execution of an ETL script. It is used like:

%remex utilities/tcia_helpers.py

remex takes a single parameter that is the path from the project root to the script to be executed

In [183]:
%alias remex ssh bcliffor@$VM env PYTHONPATH=/pycharm/etl_flow/tmp/pycharm_project_936:/pycharm/etl_flow/tmp/secure_files PYTHONUNBUFFERED=1 SECURE_LOCAL_PATH=../secure_files/etl SETTINGS_MODULE=settings python3.9 $EF/%s

### A note on logging
We are working to standardize logging as follows:

Log files are created in the directory settings.LOGGING_BASE/settings.BASE_NAME where, currently:

```
LOGGING_BASE = f'/mnt/disks/idc-etl/logs/v{CURRENT_VERSION}'
BASE_NAME = sys.argv[0].rsplit('/',1)[-1].rsplit('.',1)[0]
LOG_DIR = f'{LOGGING_BASE}/{BASE_NAME}'
```

Then we configure three loggers, successlogger, errlogger and progresslogger, which output, respectively, to success.log, error.log and progress.log files in LOG_DIR. 

In general, outputs to success.log track completed operations, and are used to avoid repeating those operations in the event that execution of a script is interrupted and must be restarted. For example, a script that copies some set of instances from one bucket to another might log the name of each blob as it is successfully copied. Then, if that script must be restarted, it can input the contents of success.log, and skip copying any instance whose ID is in that input data.

As its name implies, errors are logged to error.log.

The progresslogger is generally used to log information about the progress of the operation. 

A particular script may use all, some or none of these loggers.

Note that when a scripte is restarted, its the progress.log is truncated. The other log files are appended to.

We often execute `tail -f xxx.log` on each of the log files in separate terminal windows to monitor progress. 

## Pre-ingestion


### Update settings.py

The CURRENT_VERSION and PREVIOUS_VERSION in settings.py must be set as needed for the new version.

### Import the DB
Create a new VM, typically "idc_v\<X\>", where X is the CURRENT_VERSION in settings. Then import from the saved final idc_v\<Y\> DB where Y is PREVIOUS_VERSION.

### Revise the collection_id_map DB table
Determine if there are any non-casing collectionID changes and revise collection_id as needed. For this purpose we execute the detect_tcia_collection_name_changes.py preingestion script. Only failures are reported:

In [74]:
%remex preingestion/detect_tcia_collection_name_changes.py

[STATUS] DEBUG mode is False


A failure indicates that the TCIA ID of a collection, which we refer to as the tcia_api_collection_id, has changed. If such a change is detected, the collection_id_map table must be manually updated. 

### Update wsi metadata tables
If there is new WSI data to be ingested, we must first update the WSI database tables. There are several parameters that likely need to be specified in order to override the defaults set in the script.
We can get help to view from the build_wsi_metadata_tables_tsv.py script to view the possible parameters:

In [122]:
%remex preingestion/wsi_build/build_wsi_metadata_tables_tsv.py --help

Logging to /mnt/disks/idc-etl/logs/v10/build_wsi_metadata_tables_tsv
usage: build_wsi_metadata_tables_tsv.py [-h] [--src_bucket SRC_BUCKET]
                                        [--src_path SRC_PATH]
                                        [--tsv_blob TSV_BLOB]
                                        [--skipped_groups [SKIPPED_GROUPS ...]]
                                        [--skipped_collections [SKIPPED_COLLECTIONS ...]]

optional arguments:
  -h, --help            show this help message and exit
  --src_bucket SRC_BUCKET
                        Bucket containing WSI instances (default: htan-
                        transfer)
  --src_path SRC_PATH   Folder in src_bucket that is the root of WSI data to
                        be indexed (default:
                        HTAN-V1-Converted/Converted_20220416)
  --tsv_blob TSV_BLOB   A GCS blob that contains a TSV manifest of WSI DICOMs
                        to be ingested (default: identifiers.txt)
  --skipped_groups [SKIPPED_G

At the time of writing, a several HTAN collections have been copind into the *htan-transfer* bucket. The *HTAN-V1-Converted/Converted_20220416* folder in that bucket is the root of the specific conversion set which we want to ingest. The *identifiers.txt* blob in that folder is a tsv file that enumerates the instances in all the HTAN collections in the particular conversion. 

At this time, we only want to ingest the HTAN-OHSU collection and there want to skip the HTAN-HMS, HTAN-Vanderbilt and HTAN-WUSTL collections. None of these collections are in collection groups, all of whose members are to be skipped.

Therefore, we execute of build_wsi_metadata_tables_tsv.py:

In [113]:
%remex preingestion/wsi_build/build_wsi_metadata_tables_tsv.py \
    --src_bucket htan-transfer \
    --src_path HTAN-V1-Converted/Converted_20220416 \
    --tsv_blob identifiers.txt \
    --skipped_collections HTAN-HMS HTAN-Vanderbilt HTAN-WUSTL



Logging to /mnt/disks/idc-etl/logs/v10/build_wsi_metadata_tables_tsv
Namespace(skipped_collections=['HTAN-HMS', 'HTAN-Vanderbilt', 'HTAN-WUSTL'], skipped_groups=[], src_bucket='htan-transfer', src_path='HTAN-V1-Converted/Converted_20220416', tsv_blob='identifiers.txt')


As an aside, the preingestion/wsi_build/remove_wsi_metadata_tsv.py script does the opposite of the above: it removes any instances that is listed in a tsv and which it finds in the wsi_instance DB table. It also hierarchically removes emptied series, studies, patients and collections from the corresponding wsi tables. The parameterization is the sames as for build_wsi_metadata_tables_tsv.py. E.G., the following will remove the wsi metadata added above:

In [116]:
%remex preingestion/wsi_build/remove_wsi_metadata_tsv.py \
    --src_bucket htan-transfer \
    --src_path HTAN-V1-Converted/Converted_20220416 \
    --tsv_blob identifiers.txt \
    --skipped_collections HTAN-HMS HTAN-Vanderbilt HTAN-WUSTL


Logging to /mnt/disks/idc-etl/logs/v10/remove_wsi_metadata_tsv
Namespace(skipped_collections=['HTAN-HMS', 'HTAN-Vanderbilt', 'HTAN-WUSTL'], skipped_groups=[], src_bucket='htan-transfer', src_path='HTAN-V1-Converted/Converted_20220416', tsv_blob='identifiers.txt')


## Ingestion

In [ ]:
 We are now almost ready to ingest data. The base script is ingestion/ingest.py. It has several parameters that need to be configured:

In [125]:
%remex ingestion/ingest.py --help

Logging to /mnt/disks/idc-etl/logs/v10/ingest
usage: ingest.py [-h] [--num_processes NUM_PROCESSES]
                 [--skipped_tcia_groups SKIPPED_TCIA_GROUPS]
                 [--skipped_tcia_collections SKIPPED_TCIA_COLLECTIONS]
                 [--included_tcia_collections INCLUDED_TCIA_COLLECTIONS]
                 [--prestaging_tcia_bucket_prefix PRESTAGING_TCIA_BUCKET_PREFIX]
                 [--skipped_path_groups SKIPPED_PATH_GROUPS]
                 [--skipped_path_collections SKIPPED_PATH_COLLECTIONS]
                 [--included_path_collections INCLUDED_PATH_COLLECTIONS]
                 [--server SERVER]
                 [--prestaging_path_bucket_prefix PRESTAGING_PATH_BUCKET_PREFIX]
                 [--stop_after_collection_summary STOP_AFTER_COLLECTION_SUMMARY]

optional arguments:
  -h, --help            show this help message and exit
  --num_processes NUM_PROCESSES
                        Number of concurrent processes (default: 16)
  --skipped_tcia_groups SKIPPED_TC

ingestion/ingest.py can spawn multiple processes to speed up ingestion. If num_processes is 0, than all work is performed in the base process. We do not exceed 16 in order to not overload the TCIA/NBIA server. 

Ingestion pulls radiology data from tcia and pathology data from a bucket or buckets as specified in the wsi_instance table. Basically, ingestion asks each of the sources "What collections do you have?" and then proceeds to get new data and  update our database. However, we often want to limit the collections which we try to get from one of the sources. In particular, we do not revise excluded collections; these are collections that we have previously ingested but then did not make public because they were considered of questionable quality. Similarly, we do not revise redacted collection; these are the collections that are known to contain head scans. So we specifically do not update the radiology component of these collections, but, in some cases do want to get pathology. 

For this purpose, there are separate skipped_tcia_groups and skipped_path_groups parameters that list groups of collections which we don't want ingestion to process.

The skipped_tcia_collections and skipped_path_collections parameters allow skipping additional collections from some other collection group.

Collections identified by the include_tcia_collections and include_path_collections override corresponding sets of skipped collections. That is, the collections enumerated by these parameters are processed even if enumerated in one of the previous parameters.

The server parameter, essentially, indicates whether to check for updates to the NLST collection. Because we don't expect to ever revise NLST, this parameter can be ignored.

Ingestion copies data from the sources into per-version/per-collection/per-source buckets, e.g. idc_v10_path_tcga_brca. The prestaging_tcia_bucket_prefix and prestaging_path_bucket_prefix parameters allows changing the bucket prefix from the default `idc_v<CURRENT_VERSION>_tcia_` and `idc_v<CURRENT_VERSION>_path_`

Finally, it is often desirable to see which collections the ingestion process has identified as new, subjerct to revision, or about to be retired, before proceeding. If stop_after_collection_summary is True, ingestion will exit after printing out a summary of these pending changes:

In [132]:
%remex ingestion/ingest.py --stop_after_collection_summary True

Logging to /mnt/disks/idc-etl/logs/v10/ingest
Namespace(num_processes=16, skipped_tcia_groups=['redacted_collections', 'excluded_collections'], skipped_tcia_collections=['NLST', 'HCC-TACE-Seg'], included_tcia_collections=[], prestaging_tcia_bucket_prefix='idc_v10_tcia_', skipped_path_groups=['redacted_collections', 'excluded_collections'], skipped_path_collections=['HCC-TACE-Seg'], included_path_collections=[], server='', prestaging_path_bucket_prefix='idc_v10_', stop_after_collection_summary=True, pid=0, dicom_dir='/mnt/disks/idc-etl/dicom')
p0; Version expansion summary
p0: New collection: ACRIN-6698
p0: New collection: ISPY2
p0: Revising collection: QIN Breast DCE-MRI
p0: Revising collection: QIN LUNG CT


In the following we have added two collections to both skipped_tcia_collection and skipped_path_collections. Notice that, because the collection IDs contain spaces, we must both escape the spaces and quote the IDs

In [164]:
%remex ingestion/ingest.py \
    --skipped_tcia_collections NLST HCC-TACE-Seg 'QIN\ Breast\ DCE-MRI' 'QIN\ LUNG\ CT' \
    --skipped_path_collections NLST HCC-TACE-Seg "QIN\ Breast\ DCE-MRI"  "QIN\ LUNG\ CT" \
    --stop_after_collection_summary True

Logging to /mnt/disks/idc-etl/logs/v10/ingest
Namespace(num_processes=16, skipped_tcia_groups=['redacted_collections', 'excluded_collections'], skipped_tcia_collections=['NLST', 'HCC-TACE-Seg', 'QIN Breast DCE-MRI', 'QIN LUNG CT'], included_tcia_collections=[], prestaging_tcia_bucket_prefix='idc_v10_tcia_', skipped_path_groups=['redacted_collections', 'excluded_collections'], skipped_path_collections=['NLST', 'HCC-TACE-Seg', 'QIN Breast DCE-MRI', 'QIN LUNG CT'], included_path_collections=[], server='', prestaging_path_bucket_prefix='idc_v10_', stop_after_collection_summary=True, pid=0, dicom_dir='/mnt/disks/idc-etl/dicom')
p0; Version expansion summary
p0: New collection: ACRIN-6698
p0: New collection: ISPY2


We can now perform ingestion. This can take several days, depending on the amount of data to be ingested. In particular, pulling data from NBIA generally runs at 8-12MB/s. 

In [166]:
%remex ingestion/ingest.py \
    --num_processes 12 \
    --skipped_tcia_collections NLST HCC-TACE-Seg 'QIN\ Breast\ DCE-MRI' 'QIN\ LUNG\ CT' \
    --skipped_path_collections NLST HCC-TACE-Seg "QIN\ Breast\ DCE-MRI"  "QIN\ LUNG\ CT" 

Logging to /mnt/disks/idc-etl/logs/v10/ingest
Namespace(num_processes=12, skipped_tcia_groups=['redacted_collections', 'excluded_collections'], skipped_tcia_collections=['NLST', 'HCC-TACE-Seg', 'QIN Breast DCE-MRI', 'QIN LUNG CT'], included_tcia_collections=[], prestaging_tcia_bucket_prefix='idc_v10_tcia_', skipped_path_groups=['redacted_collections', 'excluded_collections'], skipped_path_collections=['NLST', 'HCC-TACE-Seg', 'QIN Breast DCE-MRI', 'QIN LUNG CT'], included_path_collections=[], server='', prestaging_path_bucket_prefix='idc_v10_path_', stop_after_collection_summary=False, pid=0, dicom_dir='/mnt/disks/idc-etl/dicom')
CommandException: 1 files/objects could not be removed.


### Validate UUID uniqueness
Validate that there are no collisions among all IDC generated UUIDs. 

In [168]:
%remex ingestion/validation/uuids_are_unique.py

Logging to /mnt/disks/idc-etl/logs/v10/uuids_are_unique
UUIDs are unique
INFO:root.success:UUIDs are unique


## Post-ingestion


### Revise collection group tables
There are five collection group tables in the database:
-cr_collections
-defaced_collections
-redacted_collections
-excluded_collections
These are described in the ETL Workflow document.
Any collections not in these tables are defined as open collections. open_collections is a view that resolves to the metadata of all open collections.
Unless a ingestion adds a new collection to one the cr, defaced, redacted or excludes collection groups, there is nothing to be done here; the open_collections group will include the new collection. 
Because adding new collections to any of the cr, defaced, redacted or excluded tables is expected to be done rarely, there is no script for this step. It should be done manually through the Cloud SQL CLI. Sample SQL:

```
idc_v10=> insert into cr_collections values('ISPY2', 'cc608e04-d37b-4faa-86d8-5b17068c6f80', 'idc-dev-cr', 'idc-dev-cr', 'idc-open-cr', 'idc-open-cr', 'Public', 'Public')
idc_v10=> insert into cr_collections values('ACRIN-6698', '50befbe5-9bd6-4d08-b766-b8825a8b7bb3', 'idc-dev-cr', 'idc-dev-cr', 'idc-open-cr', 'idc-open-cr', 'Public', 'Public');
```


### Revise program table
The program table associates each collection with a program. It is manually updated after updating a separately maintained [spreadsheet](https://docs.google.com/spreadsheets/d/1-sk8CMTDDj-deKv7sXglLvHUhDSNS1cRqUg5Oy5UpRY/edit#gid=0). Sample SQL:

```
idc_v10=> insert into program values ('ISPY2', 'NCI Trials');
idc_v10=> insert into program values ('ACRIN-6698', 'NCI Trials');
```


### Populate a DICOM store and export metadata to dicom_metadata BQ table
This step has several substeps.

#### Import buckets all IDC data in version
The first step is to import all instances from the idc-dev-open, idc-dev-cr, idc-dev-defaced, and idc-dev retracted staging buckets, and the premerge buckets (e.g. idc_v9_path_tcga_gbm). In other words we import into the DICOM store before merging the premerge buckets.

We will eventually export DICOM metadata to BQ. For this purpose, we import from the redacted collections because we continue to include metadat of those collections in BQ. 

The default parameters do not normally need overriding.

Note that the progress.log will show 'errors'. These are due to there being more one version of some instance in a bucket. A GCH DICOM store can only hold a single version of an instance (i.e an instance having a particular SOPInstanceUID) and reports, as an error, an attempt to import an instance having a SOPInstanceUID that is already in the DICOM store. We will deal with this issue in subsequent steps.

In [ ]:
%remex gch/populate_dicom_store/step1_import_buckets_with_redacted.py. 

#### Delete revised and retired instances from the DICOM store
The errors described above occur when there is more than one version of an instance. When such an error has occurred, we know that one version of an instance was uploaded successfully, but any other versions were rejected. We need to end up with the most recent version of any such instance, but we have no way way of knowing which version of any instance was actually imported. 

Therefore, in this substep we delete from the DICOM store, all instances that have ever been revised. In addition, by importing the entire contents of a bucket, we will have imported any instances that have been "retired"...are no longer in the IDC version. So, in this step we also.delete those retired instances.

As above, the default parameters do not normally need overriding.


In [ ]:
%remex gch/populate_dicom_store/step2_delete_revised_retired_instances_with_redacted.py

#### Insert revised instances
Now that we have deleted all instances that have revisions, we load the latest revision of each suchi instance. We do not load instances that have been retired; they are not in this IDC version.

In [ ]:
%remex gch/populate_dicom_store/step3_insert_revised_instances_with_redaction.py

#### Export DICOM metadata 
The DICOM store now holds all instances whose metadata we want to export to BQ. The step4_export_metadata.py script exports DICOM metadata to the idc-dev-etl.idc_vX_pub.dicom_metadata table. There are no parameters that might need overriding.

In [ ]:
%remex gch/populate_dicom_store/step4_export_metadata.py

#### Delete redacted instances
The DICOM store now holds instance data from redacted collections. We now remove them from so that they cannot be viewed.

In [ ]:
%remex gch/populate_dicom_store/step5_delete_redacted_instances.py

#### Validation
Validate that the DICOM store has the expected data
Validate that the dicom_metadata BQ table has the expected number of rows

In [ ]:
# TBD

### Create an external connection

In [185]:
%remex bq/bq_IO/utils/create_bq_external_connection.py \

### Upload DB tables to BQ
Next, we upload these DB tables to BQ. 
- analysis_id_map
- collection_id_map
- version
- version_collection
- collection
- collection_patient
- patient
- patient_study
- study
- study_series
- series
- series_instance
- instance
- cr_collections
- defaced_collections
- excluded_collections
- open_collections
- redacted_collections
- all_collections
- all_included_collections
- program
- non_tcia_collection_metadata'

All of these tables are uploaded by default:

In [190]:
%remex bq/bq_IO/upload_psql_to_bq.vnext.dev.py 

Logging to /mnt/disks/idc-etl/logs/v10/upload_psql_to_bq.vnext.dev
args: Namespace(federated_query='idc-dev-etl.us.etl_federated_query_idc_v10', upload=['analysis_id_map', 'collection_id_map'])
INFO:root.success:Uploading table analysis_id_map
INFO:root.success:analysis_id_map upload completed in 3.31s
INFO:root.success:Uploading table collection_id_map
INFO:root.success:collection_id_map upload completed in 2.74s


Or we can upload only selected tables e.g.:

In [193]:
%remex bq/bq_IO/upload_psql_to_bq.vnext.dev.py \
    --upload 'analysis_results_descriptions'

Logging to /mnt/disks/idc-etl/logs/v10/upload_psql_to_bq.vnext.dev
args: Namespace(federated_query='idc-dev-etl.us.etl_federated_query_idc_v10', upload=['analysis_results_descriptions'])
INFO:root.success:Uploading table analysis_results_descriptions
INFO:root.success:analysis_results_descriptions upload completed in 2.96s


Note that tables are deleted if they already exist before uploading.

### Generate analysis_results_metadata
We'll copy to the PDP staging dataset. 

In [209]:
%remex bq/gen_analysis_results_table/gen_analysis_results_metadata_table.py

Logging to /mnt/disks/idc-etl/logs/v10/gen_analysis_results_metadata_table
Namespace(bqtable_name='analysis_results_metadata')
INFO:root.success:Tue May 17 14:16:02 2022: Completed  analysis_results_metadata


### Generate original_collections_metadata
We'll copy to the PDP staging dataset. 

In [199]:
%remex bq/gen_original_data_collections_table/gen_included_original_collection_metadata.py

Namespace(bqtable_name='original_collections_metadata', gen_excluded=False, use_cached_metadata=False, cached_metadata_file='cached_included_metadata.json')
Traceback (most recent call last):
  File "/pycharm/etl_flow/tmp/pycharm_project_936/bq/gen_original_data_collections_table/gen_included_original_collection_metadata.py", line 39, in <module>
    gen_collections_table(args)
  File "/pycharm/etl_flow/tmp/pycharm_project_936/bq/gen_original_data_collections_table/gen_original_data_collection_metadata_table.py", line 387, in gen_collections_table
    metadata = build_metadata(BQ_client, args)
  File "/pycharm/etl_flow/tmp/pycharm_project_936/bq/gen_original_data_collections_table/gen_original_data_collection_metadata_table.py", line 270, in build_metadata
    collection_metadata = get_collection_metadata(client, args)
  File "/pycharm/etl_flow/tmp/pycharm_project_936/bq/gen_original_data_collections_table/gen_original_data_collection_metadata_table.py", line 234, in get_collection_met

### Generate excluded_collections_metadata
excluded_collections_metadata includes the metadata of collections in the excluded_collections group. Unlike original_collections_metadata, it is not a public table.

In [ ]:
%remex bq/gen_original_data_collections_table/gen_excluded_original_collection_metadata.py

### Generate auxiliary_metadata
The auxiliary_metadata BQ table defines the version. It contains IDC metadata for each instance in the version. In particular it includes the GCS URL of each instance.

There are two GCS blobs for each instance. One instance is in one of several buckets in the idc-dev-etl project. Another copy is (will be after Google PDP staging) in Google PDP-owned or IDC-owned public buckets. (Actually there is only a single copy, in the idc-dev-excluded bucket) of instances in the excluded collections). We maintain two versions of auxiliary_metadata: idc-dev-etl.idc_v\<version\>.auxiliary_metadata has GCS URLs in the idc-dev-etl buckets; idc-pdp-staging has GCS URLs in the public buckets. We therefore need to generate the two auxiliary_metadata metadata tables separately.

The newly ingested data remains in per-version/per-collection/per-source buckets. The following script is configured to build from data in those buckets as well as from the idc-open, idc-dev-cr, idc-dev-defaced and idc-dev-redacted staging buckets.

In [ ]:
%remex bq/gen_aux_metadata_table/gen_auxiliary_metadata_table.dev.premerge.py

Now we build the table in the idc-pdp-staging project:

In [201]:
%remex bq/gen_aux_metadata_table/gen_auxiliary_metadata_table.pdp.py

Namespace(version=10, target='pub', merged=True, dst_project='idc-pdp-staging', trg_bqdataset_name='idc_v10', bqtable_name='auxiliary_metadata', temp_license_table_name='temp_license')
Traceback (most recent call last):
  File "/pycharm/etl_flow/tmp/pycharm_project_936/bq/gen_aux_metadata_table/gen_auxiliary_metadata_table.pdp.py", line 41, in <module>
    gen_aux_table(args)
  File "/pycharm/etl_flow/tmp/pycharm_project_936/bq/gen_aux_metadata_table/gen_auxiliary_metadata_table.py", line 366, in gen_aux_table
    build_table(args)
  File "/pycharm/etl_flow/tmp/pycharm_project_936/bq/gen_aux_metadata_table/gen_auxiliary_metadata_table.py", line 362, in build_table
    result=query_BQ(client, args.trg_bqdataset_name, args.bqtable_name, query, write_disposition='WRITE_TRUNCATE')
  File "/pycharm/etl_flow/tmp/pycharm_project_936/utilities/bq_helpers.py", line 184, in query_BQ
    result = job.result()
  File "/usr/local/lib/python3.9/site-packages/google/cloud/bigquery/job/query.py", line

 ### Generate version_metadata
 Create/update a table of per-version metadata

In [211]:
%remex bq/gen_version_metadata_table/gen_version_metadata_table/py

python3.9: can't open file '/pycharm/etl_flow/tmp/pycharm_project_936/bq/gen_version_metadata_table/gen_version_metadata_table/py': [Errno 2] No such file or directory


### Copy bioclin tables

At this time the TCGA and NLST bioclinical are in the idc-dev-etl.idc_v\<version\>\_pub dataset rather than idc_clinical. They are generally unchanged across IDC versions so we just copy them from the previous version.

In [214]:
%remex bq/copy_tables/copy_bioclin_tables.vn-1_to_vn.py

Logging to /mnt/disks/idc-etl/logs/v10/copy_bioclin_tables.vn-1_to_vn
Namespace(version=10, src_project='idc-dev-etl', dst_project='idc-dev-etl', src_bqdataset='idc_v9_pub', dst_bqdataset='idc_v10_pub', dataset_description='IDC V10 BQ tables and views', bqtables=['nlst_canc', 'nlst_ctab', 'nlst_ctabc', 'nlst_prsn', 'nlst_screen', 'tcga_biospecimen_rel9', 'tcga_clinical_rel9'])
INFO:root.success:Copied table nlst_canc
INFO:root.success:Copied table nlst_ctab
INFO:root.success:Copied table nlst_ctabc
INFO:root.success:Copied table nlst_prsn
INFO:root.success:Copied table nlst_screen
INFO:root.success:Copied table tcga_biospecimen_rel9
INFO:root.success:Copied table tcga_clinical_rel9


### Copy BQ tables from dev to pdp datasets
We can now copy the following tables from idc-dev-etl.idc_v\<version\>\_pub to idc-pdp-staging.idc_v\<version\>:
- analysis_results_metadata
- dicom_metadata
- nlst_canc
- nlst_ctab
- nlst_ctabc
- nlst_prsn
- nlst_screen
- original_collections_metadata
- tcga_biospecimen_rel9
- tcga_clinical_rel9
- version_metadata

In [ ]:
%remex bq/copy_tables/copy_public_tables.dev_to_pdp.py

You can override the default --bqtables parameter, e.g.:

In [207]:
%remex bq/copy_tables/copy_public_tables.dev_to_pdp.py --bqtables nlst_canc nlst_ctab

Namespace(version=10, src_project='idc-dev-etl', dst_project='idc-pdp-staging', src_bqdataset='idc_v10_pub', dst_bqdataset='idc_v10', dataset_description='IDC V10 BQ tables and views', bqtables=['nlst_canc', 'nlst_ctab'])
Copied table nlst_canc
Copied table nlst_ctab


### Generate open_collections_blob_names
Generate a BQ table of the blob names (<uuid>.dcm) of all blobs that should be in the idc-dev-open. This is for the use of Google PDP program and is only needed in the pdp project:

In [ ]:
%remex bq/gen_open_collections_blob_names/gen_open_collections_blob_names.py

### Populate BQ views
Several BQ views are now generated. First generate views in the idc-dev-etl project:

In [ ]:
%remex view_creation/BQ_Table_Building/publish_bq_views.dev.py

Then generate views in the idc-pdp-staging project:

In [ ]:
%remex bq/view_creation/BQ_Table_Building/publish_bq_views.pdp.py

Now validate that the views "point" to tables in the expected project/dataset

In [ ]:
# TBD

### Generate idc_current dataset
There is an idc_current dataset in both the idc-dev-etl and the idc-pdp-staging projects. For completeness, these must be created (or recreated) after the webapp team has generated the dicom_derived_all table and the dicom_pivot_v\<version\> in a project. 

Generate the idc_current dataset in idc-dev-etl:

In [ ]:
%remex bq/gen_idc_current/gen_idc_current_dataset.dev.py


and in idc-pdp-staging:

In [ ]:
%remex bq/gen_idc_current/gen_idc_current_dataset.pdp.py


Now validate that the views "point" to tables in the expected project/dataset

In [ ]:
# TBD

### Generate DCF manifest
For each IDC version, we need to have DCF index all new (versions of) instances:

In [ ]:
%remex dcf/gen_instance_manifest/vX_instance_manifest.py

The resulting manifest is saved in GCS as gs://indexd_manifests/dcf_input/pdp_hosting/idc\_v\<version\>\_instance_manifest_\*.tsv, where \* is some number. The manifest for a version may be comprised of several parts. All such parts must be uploaded to [this](https://drive.google.com/drive/folders/1wNYfxLhX0Bhc_CCcuk_llB28YATX4aIm) DCF Drive folder for indexing. In addition, the new manifest name should be added to [this](https://docs.google.com/spreadsheets/d/1CcaPf4hjK9is2JbHnxpfmDYr9WJ7ZACI/edit#gid=820166513) spreadsheet.

After each manifest is indexed, we do a statistical validation. dcf/validate_indexes/validate_indexes_in_version.py takes two parameters: --version specifies the version to be validated, --starts_with is a string of hex digits (0-9a-f). The script creates a list of UUIDs of new instances in --version which start with the --starts_with value. Thus, if --starts_with is '0', then all UUIDs that start with '0' are selected, resulting in a random selection of about one in sixteen instances. A --starts_with value of '00' selects one in 256, etc.

The script then attempts resolve each GUID at the DCF server and validate that the expected URL is returned. (It only validates instances (DRS blobs), not series or studies (DRS bundles).) Resolution is quite slow, about 3/sec, so --starts_with needs to be chosen so that validation completes in some reasonable time, (whatever 'reasonable' means to the validator.)

\-\-version defaults to settings.CURRENT_VERSION  
\-\-starts_with has no default

In [217]:
%remex dcf/validate_indexes/validate_indexes_in_version.py --version 9 --starts_with '000'

Logging to /mnt/disks/idc-etl/logs/v10/validate_indexes_in_version
Namespace(version=9, starts_with='000')
INFO:root.progress:Validate 151 indexes; 
INFO:root.progress:Validating ACRIN-Contralateral-Breast-MR:0007c090-880a-4ba2-809b-90acd6f5ec53
INFO:root.success:0007c090-880a-4ba2-809b-90acd6f5ec53
INFO:root.progress:Validating ACRIN-Contralateral-Breast-MR:000caa83-3819-4c1b-9a4d-e0602e355bf1
INFO:root.success:000caa83-3819-4c1b-9a4d-e0602e355bf1
INFO:root.progress:Validating ACRIN-Contralateral-Breast-MR:00037109-1f39-4f0b-95ca-66b8d0d87a1d
INFO:root.success:00037109-1f39-4f0b-95ca-66b8d0d87a1d
INFO:root.progress:Validating ACRIN-Contralateral-Breast-MR:00030c91-8716-4adb-a922-2006464b5c4a
INFO:root.success:00030c91-8716-4adb-a922-2006464b5c4a
INFO:root.progress:Validating ACRIN-Contralateral-Breast-MR:000bab6c-67fc-481d-8907-9b17a23f9664
INFO:root.success:000bab6c-67fc-481d-8907-9b17a23f9664
INFO:root.progress:Validating ACRIN-Contralateral-Breast-MR:000a89d9-1202-4196-b4d2-095310c

INFO:root.success:000f048a-d33e-40a9-8ae2-f62e28221940
INFO:root.progress:Validating ACRIN-Contralateral-Breast-MR:00044f79-d8d4-4577-b0ba-d378f076aecd
INFO:root.success:00044f79-d8d4-4577-b0ba-d378f076aecd
INFO:root.progress:Validating ACRIN-Contralateral-Breast-MR:000be501-f2d4-40c2-8a85-053abeeb0ae7
INFO:root.success:000be501-f2d4-40c2-8a85-053abeeb0ae7
INFO:root.progress:Validating ACRIN-Contralateral-Breast-MR:000fe86c-e740-4e53-adff-e2a5f97356a1
INFO:root.success:000fe86c-e740-4e53-adff-e2a5f97356a1
INFO:root.progress:Validating ACRIN-Contralateral-Breast-MR:00007428-e9f8-47da-b16a-423f7045da97
INFO:root.success:00007428-e9f8-47da-b16a-423f7045da97
INFO:root.progress:Validating ACRIN-Contralateral-Breast-MR:000194b2-8d84-4a8a-b66f-209c6b51ba0a
INFO:root.success:000194b2-8d84-4a8a-b66f-209c6b51ba0a
INFO:root.progress:Validating ACRIN-Contralateral-Breast-MR:0007477f-81b0-4335-9a8d-03faf5d89bcf
INFO:root.success:0007477f-81b0-4335-9a8d-03faf5d89bcf
INFO:root.progress:Validating ACR

INFO:root.success:000ee723-b136-48cc-ad6f-171059b6647f
INFO:root.progress:Validating ACRIN-Contralateral-Breast-MR:0005366d-5aa7-49f2-88bd-899b3e527ac2
INFO:root.success:0005366d-5aa7-49f2-88bd-899b3e527ac2
INFO:root.progress:Validating ACRIN-Contralateral-Breast-MR:000d9a6c-33be-45b1-a31a-bfa72f82145a
INFO:root.success:000d9a6c-33be-45b1-a31a-bfa72f82145a
INFO:root.progress:Validating ACRIN-Contralateral-Breast-MR:0005ffaf-97e7-4a55-ba54-517d54973757
INFO:root.success:0005ffaf-97e7-4a55-ba54-517d54973757
INFO:root.progress:Validating ACRIN-Contralateral-Breast-MR:0003d29e-f2b1-4f7e-b3a7-e277e1cafeb1
INFO:root.success:0003d29e-f2b1-4f7e-b3a7-e277e1cafeb1
INFO:root.progress:Validating ACRIN-Contralateral-Breast-MR:00044bc1-875a-4979-a575-a72e8967a201
INFO:root.success:00044bc1-875a-4979-a575-a72e8967a201
INFO:root.progress:Validating ACRIN-Contralateral-Breast-MR:0001f26e-ba5c-4219-8d10-f8dc92321b3b
INFO:root.success:0001f26e-ba5c-4219-8d10-f8dc92321b3b
INFO:root.progress:Validating ACR

### Populate the staging and public buckets
We need to copy new instances to the staging and public buckets. 
Google PDP pulls new data from the idc-open-pdp-staging bucket. This is a “delta” bucket, containing only the instances that are new to a version. Therefore, it must be emptied before populating with new instances in the next version:

In [ ]:
%remex gcs/empty_bucket_mp/empty_pdp_bucket_mp.py

Now copy dev staging buckets to PDP staging and public buckets. The public buckets are idc-dev-cr and idc-dev-open1; these hold data from the cr, defaced, and redacted collection groups:

In [ ]:
%remex gcs/copy_new_blobs_to pub_buckets/copy_new_blobs_to pub_buckets.py.

We need to validate that the staging and public buckets are correctly populated.
Validate the staging bucket:

In [ ]:
%remex gcs/validate_bucket/validate_idc_open_pdp_staging.py

Validate idc-open-cr:

In [ ]:
%remex gcs/validate_bucket/validate_idc_open_cr.py

Validate idc-open-idc1:

In [ ]:
%remex gcs/validate_bucket/validate_public_datasets_idc.py

Check that the bigquery-public-datasets.idc_v\<version\> dataset is correct:

In [ ]:
# TBD

Check that the bigquery-public-datasets.idc_current dataset is correct:

In [ ]:
# TBD

## Post-Release

### Validate PDP release
Validate that the public-datasets-idc bucket has the correct set of instance:

In [ ]:
%remex gcs/validate_bucket/validate_idc_open_idc1.py

### Merge the premerge buckets
Up to this point, new data has remained in per-version/per-collection/per-source buckets in idc-dev-etl. We now move it to the staging buckets idc-dev-open, idc-dev-cr, idc-dev-defaced and idc-dev-redacted as needed:

In [ ]:
%remex gcs/copy_prestaging_to_staging/copy_prestaging_to_staging.py 

### Regenerate auxiliary_metadata
Now that the premerge buckets have been merged, we need to regenerate the idc-dev-etl auxiliary_metadata table so that gcs_urls are correct.

In [ ]:
%remex bq/gen_aux_metadata_table/gen_auxiliary_metadata_table.dev.postmerge.py

### Delete premerge buckets
The per-version/per-collection/per-source premerge buckets are now no longer needed.

In [ ]:
# TBD